In [1]:
import numpy as np
from numpy.random import uniform
from scipy.stats import norm, describe
import pandas as pd


In [2]:
uniform()

0.2671476279259133

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
samples_names = pd.read_csv('samples_names.csv')

In [5]:
samples_alpha = pd.read_csv('samples_alpha.csv')
samples_phi = pd.read_csv('samples_phi.csv')
samples_gamma = pd.read_csv('samples_gamma.csv')
samples_mu_beta = pd.read_csv('samples_mu_beta.csv')
samples_sigma_beta = pd.read_csv('samples_sigma_beta.csv')
samples_sigma_alpha = pd.read_csv('samples_sigma_alpha.csv')

In [6]:
adj_matrix = pd.read_csv('adj-matrix-US.csv', sep=' ')

In [7]:
def log_posterior_density(alpha, beta, gamma, theta, phi, mu_beta, sigma_beta, y):
    value = alpha + beta - gamma *(theta - phi)**2
    out = np.log(sigmoid(value)**y *(1 - sigmoid(value))**(1-y))
    dnorm = norm.logpdf(theta, 0, 1) + norm.logpdf(beta, mu_beta, sigma_beta)
    return np.sum(out + dnorm)

In [39]:
def metropolis(y, alpha_i, gamma_i, phi_i, mu_beta_i, sigma_beta_i, beta_init, theta_init, 
    iters=2000, delta=0.05, chains=2, n_warmup=1000, thin=1, verbose=False):
    
    assert(len(alpha_i) == len(gamma_i))
    assert(len(alpha_i) == len(phi_i))
    assert(len(alpha_i) == len(mu_beta_i))
    assert(len(alpha_i) == len(sigma_beta_i))
    
    samples = []
    
    for chain in range(chains):
        samples_chain = []
        
        curr = [beta_init, theta_init[chain]]
        i = 0
        for iter in range(iters):
            # getting samples from iterations
            alpha = alpha_i[iter%iters]
            gamma = gamma_i[iter%iters]
            phi = phi_i[iter%iters]
            mu_beta = mu_beta_i[iter%iters]
            sigma_beta = sigma_beta_i[iter%iters]
            
            # sampling candidate values
            cand = uniform(low= [c-delta for c in curr], high=[c+delta for c in curr])
            accept_prob = np.exp(log_posterior_density(alpha, cand[0], gamma, cand[1], phi, mu_beta, sigma_beta, y) -
                                log_posterior_density(alpha, curr[0], gamma, curr[1], phi, mu_beta, sigma_beta, y))
            
            accept_prob = np.clip(accept_prob, a_min=None, a_max=1)
            
            #print(accept_prob)
            if uniform() <= accept_prob:
                curr = cand
            
            if iter%thin == 0:
                samples_chain.append(curr)
            
        samples.append(samples_chain)
        
    return np.array(samples), describe(samples)
        

In [28]:
samples_alpha.shape

(400, 318)

In [29]:
iters=2200
np.tile(samples_alpha, (int(iters/len(samples_alpha)) + 1, 1))[:iters].shape

(2200, 318)

In [40]:
def estimation(indexes):
    beta = []
    theta = []
    for i in indexes:
        samples, _ = metropolis(y, alpha_i, gamma_i, phi_i, mu_beta_i, sigma_beta_i, np.log(np.sum(y[i,:])), np.random.normal(0, 1, size=(2)), 
            iters=3, delta=0.05, chains=2, n_warmup=0, thin=1, verbose=False)
        
        #print(samples)
        #print(np.array(samples))
        beta.append(samples[:, 0, :])
        theta.append(samples[:, 1, :])   
    
    return beta, theta

In [41]:
y = adj_matrix.as_matrix()
alpha_i = samples_alpha.as_matrix()
gamma_i = samples_gamma.as_matrix()
phi_i = samples_phi.as_matrix()
mu_beta_i = samples_mu_beta.as_matrix()
sigma_beta_i = samples_sigma_beta.as_matrix()

In [42]:
estimation([10, 11])

/Users/etienne/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in exp


([array([[ 2.04106869, -0.13758937],
         [ 2.0298087 ,  0.43963143]]), array([[ 2.90203363,  2.33407428],
         [ 2.92053058,  0.38723669]])], [array([[ 2.01966915, -0.13228866],
         [ 2.0298087 ,  0.43963143]]), array([[ 2.87122948,  2.36951903],
         [ 2.92053058,  0.38723669]])])